# Antes de empezar

conda activate python3.6_cv2

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
import os
import cv2
import skimage

import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline 

# Ignore harmless warnings:

import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import random
import string

In [2]:
import platform
print(torch.__version__)
print(platform.python_version())
torch.cuda.get_device_name(0)

1.8.1
3.6.10


'TITAN X (Pascal)'

In [3]:
letters = ['START', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
          'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'END', 'PAD']

mapeo = {}

cont = 0
for key in letters:
    vector = torch.zeros(1, 1, len(letters))
    vector[0,0,cont] = 1.0
    mapeo[key] = vector
    cont += 1

In [4]:
mapeo['START']

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [5]:
def patch_gen(word, n_patches, color_channels, patch_height, patch_width, stepsize):
    
    image = 255 * np.ones(shape = (height, width), dtype = np.uint8)
    image = cv2.putText(image, text = word, org = (5, 30),
        fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.62, color = (0, 0, 0),
        thickness = 1, lineType = cv2.LINE_AA)
    #image = skimage.util.random_noise(image, mode='s&p')
    image = transforms.ToPILImage()(image) # np.ndarray to PIL.Image.Image
    patches_tensor = torch.empty(n_patches, color_channels, patch_height, patch_width)
    patches_tensor = patches_tensor.cuda(0)
    
    for p in range(n_patches):
        
        patch = transforms.functional.crop(image, 0, 0 + p * stepsize, patch_height, patch_width) # cropping of the image into patches
        patch = transforms.ToTensor()(patch) # torch.Tensor of the patch (normalized)
        #patch = torch.from_numpy(patch) # conversion to pytorch tensor again
        patch = 1. - patch # it will work better if we have white text over black background
        patch = patch.view(1, 1, patch_height, patch_width) # CNN_model expects a 4-dimensional tensor (1 dimension for batch)
        #patch = patch.type(torch.FloatTensor) # conversion to float
        patch = patch.cuda(0) # set to cuda
        patches_tensor[p, 0, :, :] = patch
                
    return patches_tensor

In [26]:
height = 48
width = 192
patch_height = 48
patch_width = 10
stepsize = 2
color_channels = 1
batch_size=512
MAX_LENGTH = 15
n_patches = int((width - patch_width)/stepsize + 1) 

In [7]:
def complete_set(word_set):    
    complete_set = []
    
    for word in word_set:        
        complete_set.append((patch_gen(word, n_patches, color_channels, patch_height, patch_width, stepsize), word))
        
    return complete_set

In [8]:
def get_one_hot_target(labels, seq_len, output_size,batch_size):    
    one_hot_target = torch.empty(batch_size, seq_len, output_size) 

    for j,word in enumerate(labels):
        length = len(word)
        one_hot_target[j, 0, :] = mapeo['START']

        for k,letter in enumerate(word):
            one_hot_target[j, k + 1, :] = mapeo[letter]

        one_hot_target[j, length + 1, :] = mapeo['END']
        one_hot_target[j, length + 2: seq_len, :] = mapeo['PAD']
        
    return one_hot_target        

In [9]:
def one_hot_conversion(decoder_output, output_size):
    
    one_hot_output_letter = torch.zeros(1, 1, output_size).cuda(0)
    index = torch.argmax(decoder_output, dim = 2).item()
    one_hot_output_letter[0, 0, index] = 1.
    
    return one_hot_output_letter

# Definiendo la arquitectura

In [14]:
FILTROS_EN_CNN_1 = 4
NEURONS_IN_DENSE_LAYER = 1024
PATCH_HEIGHT_AFTER_POOLING = patch_height//2
PATCH_WIDTH_AFTER_POOLING = patch_width//2
kernel_size = 1

class ConvolutionalNetwork(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=FILTROS_EN_CNN_1, kernel_size=kernel_size, stride=1, padding=0)
        self.fc1 = nn.Linear(PATCH_HEIGHT_AFTER_POOLING*PATCH_WIDTH_AFTER_POOLING*FILTROS_EN_CNN_1, NEURONS_IN_DENSE_LAYER)
        
    def forward(self, X):
        X = F.relu((self.conv1(X)))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, PATCH_HEIGHT_AFTER_POOLING*PATCH_WIDTH_AFTER_POOLING*FILTROS_EN_CNN_1)
        X = self.fc1(X)

        return X

In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):        
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first = True)

    def forward(self, input, hidden, batch_size, seq_len):        
        output = input.view(batch_size, seq_len, self.input_size)
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device))

In [16]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.lstm = nn.LSTM(output_size, hidden_size, batch_first = True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim = 2)
        # dim = 2 porque esta última dimensión es la correspondiente a output_size, que es sobre
        # la que queremos hacer el softmax

    def forward(self, input, hidden, batch_size, seq_len):        
        output = input.view(batch_size, seq_len, self.output_size)
        #output = F.relu(output) # la relu se metía aquí porque en el
        #caso NLP del ejemplo de PyTorch previamente había una capa de embedding
        #No nos hace falta porque nuestro tensor de inputs ya es one-hot
        output, hidden = self.lstm(output, hidden)
        output = self.out(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
               torch.zeros(1, batch_size, self.hidden_size, device=device))

In [17]:
torch.manual_seed(1234)

encoder_input_size = 1024
hidden_size = 256
output_size = len(letters)

CNN_model = ConvolutionalNetwork().cuda(0)
CNN_optimizer = torch.optim.Adam(CNN_model.parameters())

Encoder_model = EncoderRNN(input_size = encoder_input_size, hidden_size = hidden_size).cuda(0)
Encoder_optimizer = optim.SGD(Encoder_model.parameters(), lr = 0.001)

Decoder_model = DecoderRNN(hidden_size = hidden_size, output_size = output_size).cuda(0)
Decoder_optimizer = optim.SGD(Decoder_model.parameters(), lr = 0.001)

#criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()

## Entrenando

In [18]:
import time

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [19]:
epochs = 5000
patience = 100
min_loss_val = 10 # huge initial value for the minimum validation loss

train_losses = []
val_losses = []

torch.manual_seed(1234)

In [20]:
def calculate_loss(decoder_output, ground_truth):
    loss = 0
        
    for j in range(batch_size):
        loss += criterion(decoder_output[j], ground_truth[j])               
    
    loss = loss/batch_size

    return loss

In [27]:
def train():
    
    train_losses = []
    
    for num_batch in range(0, 64):
    
        list_random_strings = []

        for i in range(batch_size):

            list_random_strings.append(''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(1,MAX_LENGTH))))

        train_set = complete_set(list_random_strings)
        train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True)

        for b, (image, labels) in enumerate(train_loader):
            
            encoder_hidden = Encoder_model.initHidden(batch_size)

            image_cnn = image.view(-1, color_channels, patch_height, patch_width).cuda(0)
            encoder_input = CNN_model(image_cnn)
            encoder_output, encoder_hidden = Encoder_model(encoder_input, encoder_hidden, batch_size = batch_size, seq_len = n_patches)

            decoder_hidden = encoder_hidden
            decoder_input = get_one_hot_target(labels=labels, batch_size = batch_size, seq_len = MAX_LENGTH + 2,output_size = output_size).cuda(0)
            decoder_output, decoder_hidden = Decoder_model(decoder_input, decoder_hidden, batch_size = batch_size, seq_len = MAX_LENGTH + 2)

            output_indices = torch.tensor(list(range(0, MAX_LENGTH + 2 -1))).cuda(0) # removing last token from the output
            decoder_output = torch.index_select(decoder_output, dim = 1, index = output_indices)

            ground_truth = torch.argmax(decoder_input, dim = 2)
            target_indices = torch.tensor(list(range(1, MAX_LENGTH + 2))).cuda(0) # remove SOS token from the input
            ground_truth = torch.index_select(ground_truth, dim = 1, index = target_indices)

            loss = calculate_loss(decoder_output,ground_truth)

            CNN_optimizer.zero_grad()
            Encoder_optimizer.zero_grad()
            Decoder_optimizer.zero_grad()
            loss.backward()
            CNN_optimizer.step()
            Encoder_optimizer.step()
            Decoder_optimizer.step()
            
            train_losses.append(loss.item())
            
        return np.mean(train_losses)

In [28]:
def validation():
    
    valid_losses = []

    with torch.no_grad():
    
        for num_batch in range(0, 16):

            list_random_strings = []

            for i in range(batch_size):

                list_random_strings.append(''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(1,MAX_LENGTH))))

            validation_set = complete_set(list_random_strings)
            val_loader = DataLoader(validation_set, batch_size = batch_size, shuffle = False)

        
            for b, (image_val, label_val) in enumerate(val_loader):        

                encoder_hidden_val = Encoder_model.initHidden(batch_size = batch_size)
                image_cnn_val = image_val.view(-1, color_channels, patch_height, patch_width).cuda(0)
                encoder_input_val = CNN_model(image_cnn_val)
                encoder_output_val, encoder_hidden_val = Encoder_model(encoder_input_val, encoder_hidden_val, batch_size = batch_size, seq_len = n_patches)

                decoder_hidden_val = encoder_hidden_val
                decoder_input_val = get_one_hot_target(labels=label_val, batch_size = batch_size, seq_len = MAX_LENGTH + 2,output_size = output_size).cuda(0)
                decoder_output_val, decoder_hidden_val = Decoder_model(decoder_input_val, decoder_hidden_val,batch_size = batch_size, seq_len = MAX_LENGTH + 2)

                output_indices_val = torch.tensor(list(range(0, MAX_LENGTH + 1))).cuda(0) # remove last token from the output
                decoder_output_val = torch.index_select(decoder_output_val, dim = 1, index = output_indices_val)

                ground_truth_val = torch.argmax(decoder_input_val, dim = 2)
                target_indices_val = torch.tensor(list(range(1, MAX_LENGTH + 2))).cuda(0) # remove START token from the input
                ground_truth_val = torch.index_select(ground_truth_val, dim = 1, index = target_indices_val)

                loss = calculate_loss(decoder_output_val,ground_truth_val)
                valid_losses.append(loss.item())
        return np.mean(valid_losses)

In [29]:
class Patience():
    
    def __init__(self, patience):
        self.patience = patience
        self.current_patience = patience
        self.min_loss_val = float('inf')

    def more_patience(self,loss_val):
        self.current_patience -= 1
        if self.current_patience == 0:
            return False

        if loss_val < self.min_loss_val:
            self.min_loss_val = loss_val
            self.current_patience = patience

            model_name = f"{height}x{width}_by{patch_width}_jump{stepsize}_batch{batch_size} NN_{FILTROS_EN_CNN_1}_{NEURONS_IN_DENSE_LAYER}_{kernel_size}_{hidden_size}_INFINITE_RANDOM_SAMPLE"
            print(", saved best model.")
            
            torch.save(CNN_model.state_dict(), 'CNN_'+model_name)
            torch.save(Encoder_model.state_dict(), 'Encoder_'+model_name)
            torch.save(Decoder_model.state_dict(), 'Decoder_'+model_name)
    
        return True

In [ ]:
torch.manual_seed(1234)

patience = 200
patience_controler = Patience(patience)
start_time = time.time()

for num_epoch in range(500000):
    train_loss = train()        
    valid_loss = validation()
    
    writer.add_scalar('Loss/train', train_loss, num_epoch)
    writer.add_scalar('Loss/validation', valid_loss, num_epoch)
    
    print(f'Epoch: {num_epoch} Train loss: {train_loss} Valid loss: {valid_loss} Duration: {(time.time() - start_time)/60} minutes',)

    if not patience_controler.more_patience(valid_loss):
        print("Se acabó la paciencia")
        break


Epoch: 0 Train loss: 3.376540184020996 Valid loss: 3.3753931671380997 Duration: 1.8478599707285563 minutes
, saved best model.
Epoch: 1 Train loss: 3.375528335571289 Valid loss: 3.374031573534012 Duration: 3.6963781078656512 minutes
, saved best model.
Epoch: 2 Train loss: 3.373711585998535 Valid loss: 3.3726245164871216 Duration: 5.540303285916647 minutes
, saved best model.
Epoch: 3 Train loss: 3.37314772605896 Valid loss: 3.3715538680553436 Duration: 7.390025305747986 minutes
, saved best model.
Epoch: 4 Train loss: 3.3716237545013428 Valid loss: 3.370556116104126 Duration: 9.23621216615041 minutes
, saved best model.
Epoch: 5 Train loss: 3.3712573051452637 Valid loss: 3.3695359230041504 Duration: 11.081617279847462 minutes
, saved best model.
Epoch: 6 Train loss: 3.369041919708252 Valid loss: 3.368569076061249 Duration: 12.924023954073588 minutes
, saved best model.
Epoch: 7 Train loss: 3.3684046268463135 Valid loss: 3.3674750477075577 Duration: 14.769381566842396 minutes
, saved b

Epoch: 66 Train loss: 3.2614071369171143 Valid loss: 3.2560299932956696 Duration: 123.94006844758988 minutes
, saved best model.
Epoch: 67 Train loss: 3.2583107948303223 Valid loss: 3.2573185116052628 Duration: 125.78571435610453 minutes
Epoch: 68 Train loss: 3.256436824798584 Valid loss: 3.254881963133812 Duration: 127.63258495330811 minutes
, saved best model.
Epoch: 69 Train loss: 3.2501394748687744 Valid loss: 3.2534548342227936 Duration: 129.4792286435763 minutes
, saved best model.
Epoch: 70 Train loss: 3.258434772491455 Valid loss: 3.2517505884170532 Duration: 131.32735763788224 minutes
, saved best model.
Epoch: 71 Train loss: 3.2547192573547363 Valid loss: 3.251824975013733 Duration: 133.17536105712256 minutes
Epoch: 72 Train loss: 3.2496731281280518 Valid loss: 3.2492272555828094 Duration: 135.02197453975677 minutes
, saved best model.
Epoch: 73 Train loss: 3.253000020980835 Valid loss: 3.250415727496147 Duration: 136.8662702004115 minutes
Epoch: 74 Train loss: 3.233665704727

Epoch: 135 Train loss: 3.189509153366089 Valid loss: 3.1994022876024246 Duration: 251.36976888577144 minutes
Epoch: 136 Train loss: 3.182701349258423 Valid loss: 3.1975756734609604 Duration: 253.21848467985788 minutes
Epoch: 137 Train loss: 3.1799941062927246 Valid loss: 3.195961594581604 Duration: 255.06802299817403 minutes
Epoch: 138 Train loss: 3.220182180404663 Valid loss: 3.191705137491226 Duration: 256.915893137455 minutes
, saved best model.
Epoch: 139 Train loss: 3.2061257362365723 Valid loss: 3.1920445263385773 Duration: 258.7643669644992 minutes
Epoch: 140 Train loss: 3.1863276958465576 Valid loss: 3.1923710107803345 Duration: 260.6100320617358 minutes
Epoch: 141 Train loss: 3.1696221828460693 Valid loss: 3.1847199201583862 Duration: 262.45689551830293 minutes
, saved best model.
Epoch: 142 Train loss: 3.1983001232147217 Valid loss: 3.1936338245868683 Duration: 264.30527269442877 minutes
Epoch: 143 Train loss: 3.193655252456665 Valid loss: 3.1920854598283768 Duration: 266.152

Epoch: 207 Train loss: 3.13274884223938 Valid loss: 3.149019256234169 Duration: 384.3481112599373 minutes
Epoch: 208 Train loss: 3.1421282291412354 Valid loss: 3.1489887684583664 Duration: 386.1926651159922 minutes
Epoch: 209 Train loss: 3.157188892364502 Valid loss: 3.1452592462301254 Duration: 388.037515215079 minutes
, saved best model.
Epoch: 210 Train loss: 3.1479272842407227 Valid loss: 3.144139811396599 Duration: 389.8833063840866 minutes
, saved best model.
Epoch: 211 Train loss: 3.1527867317199707 Valid loss: 3.139383226633072 Duration: 391.73087379932406 minutes
, saved best model.
Epoch: 212 Train loss: 3.136479377746582 Valid loss: 3.1453002393245697 Duration: 393.5755486885707 minutes
Epoch: 213 Train loss: 3.1396279335021973 Valid loss: 3.14705428481102 Duration: 395.42091976801555 minutes
Epoch: 214 Train loss: 3.152351140975952 Valid loss: 3.147614896297455 Duration: 397.26717706918714 minutes
Epoch: 215 Train loss: 3.155599355697632 Valid loss: 3.1433197259902954 Durat

Epoch: 280 Train loss: 3.1051852703094482 Valid loss: 3.10958893597126 Duration: 519.1537463267645 minutes
Epoch: 281 Train loss: 3.101224660873413 Valid loss: 3.1031460911035538 Duration: 520.996068139871 minutes
Epoch: 282 Train loss: 3.1026952266693115 Valid loss: 3.0989574640989304 Duration: 522.837954934438 minutes
Epoch: 283 Train loss: 3.104508876800537 Valid loss: 3.103582486510277 Duration: 524.6807196855545 minutes
Epoch: 284 Train loss: 3.127877712249756 Valid loss: 3.1109063923358917 Duration: 526.5224078814189 minutes
Epoch: 285 Train loss: 3.0992085933685303 Valid loss: 3.098812773823738 Duration: 528.3654521743457 minutes
Epoch: 286 Train loss: 3.0845866203308105 Valid loss: 3.1005632132291794 Duration: 530.2122662146886 minutes
Epoch: 287 Train loss: 3.0982649326324463 Valid loss: 3.1044293493032455 Duration: 532.0592505852381 minutes
Epoch: 288 Train loss: 3.0885558128356934 Valid loss: 3.1029745787382126 Duration: 533.9034483790398 minutes
Epoch: 289 Train loss: 3.086

Epoch: 354 Train loss: 3.0770821571350098 Valid loss: 3.064258188009262 Duration: 655.8030522068342 minutes
Epoch: 355 Train loss: 3.086073160171509 Valid loss: 3.066983073949814 Duration: 657.6505787650744 minutes
Epoch: 356 Train loss: 3.056857109069824 Valid loss: 3.0677777230739594 Duration: 659.495664858818 minutes
Epoch: 357 Train loss: 3.057933807373047 Valid loss: 3.0614647269248962 Duration: 661.3401191194852 minutes
, saved best model.
Epoch: 358 Train loss: 3.059025526046753 Valid loss: 3.0653997510671616 Duration: 663.1827475627264 minutes
Epoch: 359 Train loss: 3.056196451187134 Valid loss: 3.059045895934105 Duration: 665.0259864648183 minutes
, saved best model.
Epoch: 360 Train loss: 3.086130380630493 Valid loss: 3.066851496696472 Duration: 666.8687577724456 minutes
Epoch: 361 Train loss: 3.061101198196411 Valid loss: 3.059284895658493 Duration: 668.710777759552 minutes
Epoch: 362 Train loss: 3.057584524154663 Valid loss: 3.062182918190956 Duration: 670.5534772396088 min

Epoch: 427 Train loss: 3.0468621253967285 Valid loss: 3.0303279012441635 Duration: 790.4880872050921 minutes
Epoch: 428 Train loss: 3.0537588596343994 Valid loss: 3.0204939395189285 Duration: 792.3331376949947 minutes
, saved best model.
Epoch: 429 Train loss: 3.0227878093719482 Valid loss: 3.027157962322235 Duration: 794.1774938583374 minutes
Epoch: 430 Train loss: 3.038226842880249 Valid loss: 3.023071527481079 Duration: 796.0228035767873 minutes
Epoch: 431 Train loss: 2.9999332427978516 Valid loss: 3.023884281516075 Duration: 797.8690736611684 minutes
Epoch: 432 Train loss: 3.0019357204437256 Valid loss: 3.024831086397171 Duration: 799.7151622494061 minutes
Epoch: 433 Train loss: 3.027595043182373 Valid loss: 3.0183248221874237 Duration: 801.5632421135903 minutes
, saved best model.
Epoch: 434 Train loss: 2.9944024085998535 Valid loss: 3.0212394297122955 Duration: 803.4087690830231 minutes
Epoch: 435 Train loss: 3.0289394855499268 Valid loss: 3.0190470069646835 Duration: 805.2561062

## Cargar modelo

In [ ]:
CNN_model.load_state_dict(torch.load('CNN_model_30000_words_TF_PAD_noise.pt'))
CNN_model.eval()

Encoder_model.load_state_dict(torch.load('Encoder_model_30000_words_TF_PAD_noise.pt'))
Encoder_model.eval()

Decoder_model.load_state_dict(torch.load('Decoder_model_30000_words_TF_PAD_noise.pt'))
Decoder_model.eval()

In [3]:
def Test():
    with torch.no_grad():
        for num_batch, (image_test, label_test) in enumerate(test_loader):
            num_batch += 1
            encoder_hidden_test = Encoder_model.initHidden(batch_size = batch_size)
            image_cnn_test = image_test.view(-1, color_channels, patch_height, patch_width).cuda(0)
            encoder_input_test = CNN_model(image_cnn_test)
            encoder_output, encoder_hidden_test = Encoder_model(encoder_input_test, encoder_hidden_test, batch = test_batch, seq_len = n_patches)

            #decoder_hidden_test = (encoder_hidden_test[0][0, :, :].view(1, batch_size, hidden_size), # We take the last hidden state of the Encoder 
            #                       encoder_hidden_test[1][0, :, :].view(1, batch_size, hidden_size)) # for each image/word (j) within the batch 
            
            for j in range(batch_size):
                decoder_input_test = mapeo['START'].cuda(0) # We initialize the first Decoder input as the START token
                decoder_hidden_test = (encoder_hidden_test[0][0, j, :].view(1, 1, hidden_size), # We take the last hidden state of the Encoder 
                                       encoder_hidden_test[1][0, j, :].view(1, 1, hidden_size)) # for each image/word (j) within the batch 
                
                for d in range(MAX_LENGTH + 2):
                    decoder_output_test, decoder_hidden_test = Decoder_model(decoder_input_test, decoder_hidden_test, batch = 1, seq_len = 1)

                    output_letter = one_hot_conversion(decoder_output_test, output_size = output_size)
                    decoder_input_test = output_letter
                    
                    if d == 0:
                        output_word = output_letter
                    else:
                        output_word = torch.cat((output_word, output_letter), dim = 1).cuda(0)
                    
                    if torch.equal(output_letter, letter_to_vector('END').cuda(0)):
                        break
                output_word = torch.argmax(output_word, dim=2)
                output_word = output_word.view(output_word.numel()) # view as a rank-1 tensor

                model_word = []
                for item in output_word:
                    model_word.append(letters[item])

                model_word = ''.join(model_word[:-1])
                print(model_word)
            print(test_set) 


In [4]:
Test()

NameError: name 'torch' is not defined